## 1. Census Data

In [1]:
import pandas as pd
import re

In [2]:
cens = pd.read_csv('2016Census_G01_VIC_SSC.csv')
cens = cens.iloc[:,[0,3]]
# Change column names
cens.columns= ['SSC','TotPop']
# Delete rows with 0 population
cens = cens.loc[cens.TotPop!=0]
# Delete "SSC" in the first column
i=0
while i < cens.shape[0]:
    cens.iloc[i,0] = cens.iloc[i,0][3:]
    i = i+1
cens.SSC = cens.SSC.astype('int64')

In [3]:
# Drop the record with 22543
cens.drop(index=cens[cens.SSC==22543].index,inplace=True)

In [4]:
cens.head()

,SSC,TotPop
1,20002,8184
2,20003,3897
4,20005,171
6,20007,59
7,20008,71


### SSC Data

In [5]:
ssc=pd.read_csv('SSC_2016_AUST.csv')
ssc=ssc[ssc.STATE_NAME_2016=='Victoria'].groupby(['SSC_CODE_2016','SSC_NAME_2016']).sum()
ssc.reset_index(inplace=True)
ssc=ssc[['SSC_CODE_2016','SSC_NAME_2016']]
ssc.columns=['SSC','Suburb']
ssc.head()

,SSC,Suburb
0,20001,Abbeyard
1,20002,Abbotsford (Vic.)
2,20003,Aberfeldie
3,20004,Aberfeldy
4,20005,Acheron


In [6]:
for i in range(len(ssc)):
    ssc.iloc[i,1]=re.match(r'[a-zA-Z ]*', ssc.iloc[i,1]).group(0)
    ssc.iloc[i,1]=ssc.iloc[i,1].strip(' ')
cens.SSC = cens.SSC.astype('int64')

In [7]:
print(ssc.shape)
print(cens.shape)

(2931, 2)
(2817, 2)


In [9]:
ssc.head()

,SSC,Suburb
0,20001,Abbeyard
1,20002,Abbotsford
2,20003,Aberfeldie
3,20004,Aberfeldy
4,20005,Acheron


### Merge Census and SSC

In [10]:
sub_pop = pd.merge(ssc, cens, on='SSC', how='inner')
sub_pop = sub_pop[['Suburb','TotPop']]
sub_pop.Suburb=sub_pop.Suburb.str.upper()
sub_pop.head()

,Suburb,TotPop
0,ABBOTSFORD,8184
1,ABERFELDIE,3897
2,ACHERON,171
3,ADAMS ESTATE,59
4,ADDINGTON,71


In [11]:
sub_pop[sub_pop['Suburb']=='TOTTENHAM']

,Suburb,TotPop


In [12]:
sub_pop.head()

,Suburb,TotPop
0,ABBOTSFORD,8184
1,ABERFELDIE,3897
2,ACHERON,171
3,ADAMS ESTATE,59
4,ADDINGTON,71


## 2. Crime Data

In [15]:
off = pd.read_csv('Offence Data.csv')

In [16]:
off = off.loc[:,['Year ending December','Postcode','Suburb/Town Name','Offence Count']]
# change column names
off.columns=['Year','Postcode','Suburb','OffenceCount']
# filter data with year 2018
off = off[off.Year == 2018]
off=off[['Postcode','Suburb','OffenceCount']]
# Replace ',' with '' in OffenceCount
off.OffenceCount = pd.Series(off.OffenceCount).str.replace(',','',regex=False)
# Change datatype of OffenceCount column
off.OffenceCount = pd.to_numeric(off.OffenceCount)
off.head()

,Postcode,Suburb,OffenceCount
30412,3000,MELBOURNE,709
30413,3000,MELBOURNE,135
30414,3000,MELBOURNE,569
30415,3000,MELBOURNE,114
30416,3000,MELBOURNE,187


In [17]:
off = off.groupby(['Suburb','Postcode'])[['OffenceCount']].sum()
off.reset_index(inplace=True)
off.head()

,Suburb,Postcode,OffenceCount
0,ABBEYARD,3737,2
1,ABBOTSFORD,3067,1478
2,ABERFELDIE,3040,97
3,ACHERON,3714,5
4,ADDINGTON,3352,4


In [20]:
off.shape

(2424, 3)

## 3. Crime Rate

In [21]:
pop=sub_pop
print(off.shape)
print(pop.shape)

(2424, 3)
(2817, 2)


In [22]:
# Check how many suburbs in offence are not in the population table.
i = 0
excep_list=[]
while i < off.shape[0]:
    if off.iloc[i,0] not in list(pop.Suburb):
#         print('offence data suburb', off.iloc[i,0], 'is not in census')
        excep_list.append(off.iloc[i,0])
    i = i+1
print(len(excep_list))

148


In [23]:
pop[pop['Suburb']=='WOODEND']

,Suburb,TotPop
2713,WOODEND,5806


In [24]:
merge = pd.merge(off, pop, on='Suburb', how='right')
merge.head()

,Suburb,Postcode,OffenceCount,TotPop
0,ABBOTSFORD,3067.0,1478.0,8184
1,ABERFELDIE,3040.0,97.0,3897
2,ACHERON,3714.0,5.0,171
3,ADDINGTON,3352.0,4.0,71
4,ADELAIDE LEAD,3465.0,3.0,81


In [25]:
merge.shape

(2830, 4)

In [26]:
merge=pd.DataFrame.drop_duplicates(merge)

In [27]:
merge.shape

(2830, 4)

In [28]:
merge['CrimeRate']= round(merge.OffenceCount/merge.TotPop,4)
merge.head()

,Suburb,Postcode,OffenceCount,TotPop,CrimeRate
0,ABBOTSFORD,3067.0,1478.0,8184,0.1806
1,ABERFELDIE,3040.0,97.0,3897,0.0249
2,ACHERON,3714.0,5.0,171,0.0292
3,ADDINGTON,3352.0,4.0,71,0.0563
4,ADELAIDE LEAD,3465.0,3.0,81,0.0370


In [37]:
# merge.to_csv('2018CrimeRate_Data.csv',index = False)

In [30]:
merge[merge['TotPop'].isnull()].shape

(0, 5)

In [32]:
merge[merge['OffenceCount'].isnull()].shape

(538, 5)

In [29]:
merge[merge['Suburb']=='MALVERN']

,Suburb,Postcode,OffenceCount,TotPop,CrimeRate
1244,MALVERN,3144.0,760.0,10066,0.0755


In [34]:
# List suburbs without data
suburb_withoutData=merge[merge.isnull().any(axis=1)].Suburb.tolist()
suburb_withoutData

['ADAMS ESTATE',
 'AGNES',
 'ALLAMBEE SOUTH',
 'ANNUELLO',
 'APPIN',
 'ARCHIES CREEK',
 'AREEGRA',
 'ARNOLD WEST',
 'AUBREY',
 'AVONMORE',
 'BAEL BAEL',
 'BAGSHOT NORTH',
 'BALINTORE',
 'BALLAPUR',
 'BALOOK',
 'BAMGANIE',
 'BANYAN',
 'BARINGHUP WEST',
 'BARKLY',
 'BARKSTEAD',
 'BAROMI',
 'BARRAPORT WEST',
 'BARUNAH PLAINS',
 'BARWIDGEE',
 'BASALT',
 'BATHUMI',
 'BAYINDEEN',
 'BEAZLEYS BRIDGE',
 'BEENAK',
 'BELLELLEN',
 'BENDOC',
 'BENNISON',
 'BERRIMAL',
 'BETE BOLONG NORTH',
 'BIG DESERT',
 'BIG PATS CREEK',
 'BIRCHIP WEST',
 'BLACKWARRY',
 'BLOWHARD',
 'BOCHARA',
 'BOGONG',
 'BOHO SOUTH',
 'BOIGBEAT',
 'BOINKA',
 'BOLTON',
 'BOOLARONG',
 'BOOLITE',
 'BOOMAHNOOMOONAH',
 'BOONAH',
 'BOORCAN',
 'BOORHAMAN EAST',
 'BOOROOL',
 'BORNES HILL',
 'BOSTOCKS CREEK',
 'BOWMANS FOREST',
 'BOXWOOD',
 'BRADVALE',
 'BREAKAWAY CREEK',
 'BREWSTER',
 'BRIDGEWATER NORTH',
 'BRIMBOAL',
 'BRINGALBERT',
 'BRIT BRIT',
 'BROOKVILLE',
 'BROUGHTON',
 'BRUARONG',
 'BRUMBY',
 'BUCKLEY SWAMP',
 'BUDGERUM EAST',
 

## 4. Geojson Data

In [30]:
import json

In [32]:
geo = open('geojson_simplify2.json')
geo = json.load(geo)
type(geo)

dict

In [34]:
geo_str = json.dumps(geo)
cr=merge
mel_suburb = list(cr.Suburb)
len(mel_suburb)

2830

In [37]:
feature_list = geo['features']
print("There're",len(feature_list),"suburbs included in the geojson data")

There're 2973 suburbs included in the geojson data


In [38]:
location_dic = {'Suburb':[],'Geometry':[]}
for item in feature_list:
    location_dic['Suburb'].append(item['properties']['vic_loca_2'].title())
    location_dic['Geometry'].append(str(item['geometry']))
suburb_geo = pd.DataFrame(location_dic)
suburb_geo.Suburb=suburb_geo.Suburb.str.upper()
suburb_geo.head()

,Suburb,Geometry
0,UNDERBOOL,"{'type': 'Polygon', 'coordinates': [[[141.7455..."
1,NURRAN,"{'type': 'Polygon', 'coordinates': [[[148.6687..."
2,WOORNDOO,"{'type': 'Polygon', 'coordinates': [[[142.9228..."
3,DEPTFORD,"{'type': 'Polygon', 'coordinates': [[[147.8233..."
4,YANAC,"{'type': 'Polygon', 'coordinates': [[[141.2797..."


In [39]:
suburb_geo.shape

(2973, 2)

### Merge CrimeRate and Suburb_geo

In [40]:
cr.head()

,Suburb,Postcode,OffenceCount,TotPop,CrimeRate
0,ABBOTSFORD,3067.0,1478.0,8184,0.1806
1,ABERFELDIE,3040.0,97.0,3897,0.0249
2,ACHERON,3714.0,5.0,171,0.0292
3,ADDINGTON,3352.0,4.0,71,0.0563
4,ADELAIDE LEAD,3465.0,3.0,81,0.0370


In [41]:
result = pd.merge(cr, suburb_geo, on='Suburb', how='inner')
result.head()

,Suburb,Postcode,OffenceCount,TotPop,CrimeRate,Geometry
0,ABBOTSFORD,3067.0,1478.0,8184,0.1806,"{'type': 'Polygon', 'coordinates': [[[144.9913..."
1,ABERFELDIE,3040.0,97.0,3897,0.0249,"{'type': 'Polygon', 'coordinates': [[[144.9015..."
2,ACHERON,3714.0,5.0,171,0.0292,"{'type': 'Polygon', 'coordinates': [[[145.6416..."
3,ADDINGTON,3352.0,4.0,71,0.0563,"{'type': 'Polygon', 'coordinates': [[[143.6572..."
4,ADELAIDE LEAD,3465.0,3.0,81,0.0370,"{'type': 'Polygon', 'coordinates': [[[143.7116..."


In [42]:
result.shape

(2868, 6)

In [43]:
# Delete records with duplicates names
dup_index = result[result.duplicated(['Suburb'])].index
result.drop(index=dup_index,inplace=True)

In [44]:
result.shape

(2801, 6)

In [45]:
result[result.isnull().any(axis=1)]

,Suburb,Postcode,OffenceCount,TotPop,CrimeRate,Geometry
2333,ADAMS ESTATE,NaN,NaN,59,NaN,"{'type': 'Polygon', 'coordinates': [[[145.5291..."
2334,AGNES,NaN,NaN,57,NaN,"{'type': 'Polygon', 'coordinates': [[[146.3995..."
2335,ALLAMBEE SOUTH,NaN,NaN,76,NaN,"{'type': 'Polygon', 'coordinates': [[[146.0755..."
2336,ANNUELLO,NaN,NaN,25,NaN,"{'type': 'Polygon', 'coordinates': [[[142.9144..."
2337,APPIN,NaN,NaN,11,NaN,"{'type': 'Polygon', 'coordinates': [[[143.9185..."
2338,ARCHIES CREEK,NaN,NaN,22,NaN,"{'type': 'Polygon', 'coordinates': [[[145.5791..."
2339,AREEGRA,NaN,NaN,35,NaN,"{'type': 'Polygon', 'coordinates': [[[142.7280..."
2340,ARNOLD WEST,NaN,NaN,22,NaN,"{'type': 'Polygon', 'coordinates': [[[143.7965..."
2341,AUBREY,NaN,NaN,10,NaN,"{'type': 'Polygon', 'coordinates': [[[142.2698..."
2342,AVONMORE,NaN,NaN,43,NaN,"{'type': 'Polygon', 'coordinates': [[[144.4693..."


In [46]:
result.head()

,Suburb,Postcode,OffenceCount,TotPop,CrimeRate,Geometry
0,ABBOTSFORD,3067.0,1478.0,8184,0.1806,"{'type': 'Polygon', 'coordinates': [[[144.9913..."
1,ABERFELDIE,3040.0,97.0,3897,0.0249,"{'type': 'Polygon', 'coordinates': [[[144.9015..."
2,ACHERON,3714.0,5.0,171,0.0292,"{'type': 'Polygon', 'coordinates': [[[145.6416..."
3,ADDINGTON,3352.0,4.0,71,0.0563,"{'type': 'Polygon', 'coordinates': [[[143.6572..."
4,ADELAIDE LEAD,3465.0,3.0,81,0.0370,"{'type': 'Polygon', 'coordinates': [[[143.7116..."


In [48]:
result.to_csv('2018CrimeRate_Data.csv',index = False)

## Camera Data

In [58]:
import json
import re

In [10]:
camera = open('camera.json')
camera = json.load(camera)
type(camera)


list

In [11]:
camera

[{'attributes': {'FID': 63,
   'OBJECTID': None,
   'MCC_ID': None,
   'STR_ID': None,
   'ASSET_CLAS': None,
   'Title': 'Safe City Camera',
   'ASSET_SUBT': None,
   'Location': 'Security - Safe City - Camera 33: Corner Collins and Exhibition Street',
   'MODEL_NO': None,
   'MODEL_DESC': None,
   'Content': '',
   'PROP_ID': None,
   'ROADSEG_ID': None,
   'ADDRESSPT_': None,
   'ADDRESSPT1': None,
   'ADDRESSP_1': None},
  'geometry': {'x': 16138174.729999024, 'y': -4553193.473842271}},
 {'attributes': {'FID': 64,
   'OBJECTID': None,
   'MCC_ID': None,
   'STR_ID': None,
   'ASSET_CLAS': None,
   'Title': 'Safe City Camera',
   'ASSET_SUBT': None,
   'Location': 'Security - Safe City - Camera 24:Bourke Street Mall',
   'MODEL_NO': None,
   'MODEL_DESC': None,
   'Content': '',
   'PROP_ID': None,
   'ROADSEG_ID': None,
   'ADDRESSPT_': None,
   'ADDRESSPT1': None,
   'ADDRESSP_1': None},
  'geometry': {'x': 16137372.541319622, 'y': -4553172.357648708}},
 {'attributes': {'FID': 65,

In [57]:
list(camera[0]['geometry'].values())

[16138174.729999024, -4553193.473842271]

In [44]:
camera_list = []
for record in camera:
    camera_list.append(record['geometry'])
camera_dic['']

62

In [45]:
st = []
for record in camera:
    st.append(record['attributes']['Location'])
for i in range(len(st)):
    st[i]=re.sub(r'Security - Safe City - Camera [#\d -:]*','',st[i])
    st[i]=re.sub(r'Camera [#\d -:]*','',st[i])
    st[i]=st[i].strip(' ')
st.remove('')
st

['Corner Collins and Exhibition Street',
 'Bourke Street Mall',
 'Bourke Street & King',
 'Corner Queen and Little Collins Street',
 'Crown Sign on King Street Bridge',
 'Corner Flinders La/Staughton Ally',
 'Cnr Spencer St/Lonsdale St',
 'Little Bourke Street & Russell Street',
 'Corner Latrobe St/ Harbour Esp',
 'Spencer St (Batman Park)',
 'Flinders Lane & Swanston Street',
 'Corner Latrobe and Swanston Street',
 'Corner Finders Lane and Elizabeth Street',
 'Riverside/St Kilds Road Bridge',
 'Little Collins Street & Swanston Street',
 'Harbour ESP SE Corner West Wharf',
 'Security - Safe City Latrobe and Elizabeth',
 'Outside 55 King Street',
 'Corner Latrobe/William',
 'Bourke Street & Russell Street',
 'Flinders Street Station steps',
 'Flinders Street & Elizabeth Street',
 'Harbour Esp SE Crn Center Wharf',
 'Flinders Street & Swanston Street',
 'Collins Street & King Street',
 'Little Collins Street & King Street',
 'Bourke Street & Swanston Street',
 'Clarendon Street Opposite 

In [46]:
len(st)

61

In [48]:
for i in range(len(st)):
    st[i]=re.sub(r'Corner','',st[i])
    st[i]=re.sub(r'Outside','',st[i])
    st[i] = st[i].strip(' ')
st

['Collins and Exhibition Street',
 'Bourke Street Mall',
 'Bourke Street & King',
 'Queen and Little Collins Street',
 'Crown Sign on King Street Bridge',
 'Flinders La/Staughton Ally',
 'Cnr Spencer St/Lonsdale St',
 'Little Bourke Street & Russell Street',
 'Latrobe St/ Harbour Esp',
 'Spencer St (Batman Park)',
 'Flinders Lane & Swanston Street',
 'Latrobe and Swanston Street',
 'Finders Lane and Elizabeth Street',
 'Riverside/St Kilds Road Bridge',
 'Little Collins Street & Swanston Street',
 'Harbour ESP SE  West Wharf',
 'Security - Safe City Latrobe and Elizabeth',
 '55 King Street',
 'Latrobe/William',
 'Bourke Street & Russell Street',
 'Flinders Street Station steps',
 'Flinders Street & Elizabeth Street',
 'Harbour Esp SE Crn Center Wharf',
 'Flinders Street & Swanston Street',
 'Collins Street & King Street',
 'Little Collins Street & King Street',
 'Bourke Street & Swanston Street',
 'Clarendon Street Opposite Convention Center',
 'Queen and Flinders Street',
 'Lonsdale St

## Street Light Data

In [49]:
import json
light = open('StreetLight.geojson')
light = json.load(light)
type(light)

dict

In [50]:
light = light['features']
len(light)

106037

In [57]:
result = []
for i in range(len(light)):
    temp = []
    temp.append(light[i]['geometry']['coordinates'][1])
    temp.append(light[i]['geometry']['coordinates'][0])
    temp.append(light[i]['properties']['label'])
    result.append(temp)

In [58]:
len(result)

106037

#### Streelight Shorten V1 1/10

In [74]:
# Shorten version 1
short1 = []
for i in range(len(result)):
    if i%10 == 0:
        short1.append(result[i])
short_json = []
for record in short1:
    temp = {}
    temp['lat'] = round(record[0],6)
    temp['log'] = round(record[1],6)
    temp['lux'] = float(record[2])
    short_json.append(temp)
short_json = json.dumps(short_json)
file = open('LightLocation_short1.json','w')
file.write(short_json)
file.close()

In [77]:
len(short1)

10604

#### Streetlight Shorten V2 1/100

In [78]:
# Shorten version 1
short2 = []
for i in range(len(result)):
    if i%100 == 0:
        short2.append(result[i])
short_json = []
for record in short2:
    temp = {}
    temp['lat'] = round(record[0],6)
    temp['log'] = round(record[1],6)
    temp['lux'] = float(record[2])
    short_json.append(temp)
short_json = json.dumps(short_json)
file = open('LightLocation_short2.json','w')
file.write(short_json)
file.close()

In [79]:
len(short2)

1061

In [88]:
result_json = []
for record in result:
    temp = {}
    temp['lat'] = round(record[0],6)
    temp['log'] = round(record[1],6)
    temp['lux'] = float(record[2])
    result_json.append(temp)
result_json = json.dumps(result_json)
file = open('LightLocation.json','w')
file.write(result_json)
file.close()

[{'lat': -37.813739, 'log': 144.942129, 'lux': 6.549},
 {'lat': -37.813744, 'log': 144.942139, 'lux': 6.158},
 {'lat': -37.813803, 'log': 144.94222, 'lux': 54.448},
 {'lat': -37.813805, 'log': 144.942225, 'lux': 58.553},
 {'lat': -37.81381, 'log': 144.94224, 'lux': 98.143},
 {'lat': -37.813816, 'log': 144.942253, 'lux': 97.165},
 {'lat': -37.813515, 'log': 144.941788, 'lux': 21.799},
 {'lat': -37.813519, 'log': 144.941793, 'lux': 18.28},
 {'lat': -37.813556, 'log': 144.941821, 'lux': 23.754},
 {'lat': -37.813566, 'log': 144.941828, 'lux': 34.409},
 {'lat': -37.813578, 'log': 144.941845, 'lux': 45.943},
 {'lat': -37.813619, 'log': 144.941913, 'lux': 2.542},
 {'lat': -37.813636, 'log': 144.941945, 'lux': 2.151},
 {'lat': -37.813637, 'log': 144.941948, 'lux': 2.151},
 {'lat': -37.813643, 'log': 144.941961, 'lux': 3.812},
 {'lat': -37.81365, 'log': 144.941973, 'lux': 9.58},
 {'lat': -37.813651, 'log': 144.941976, 'lux': 13.099},
 {'lat': -37.813658, 'log': 144.941987, 'lux': 27.37},
 {'lat